In [236]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [237]:
cd workspace/theislab/mubind/notebooks/batch

[Errno 2] No such file or directory: 'workspace/theislab/mubind/notebooks/batch'
/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks/batch


In [238]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.Mubind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

Using device: cuda:0


In [239]:
import os

df = []
basedir = '../../../mubind-pipeline/output/snakemake/SELEX'
batch = 0
for tf in os.listdir(basedir):
    for f in os.listdir(os.path.join(basedir, tf)):
        if f.endswith('.tsv.gz'):
            df2 = pd.read_csv(os.path.join(basedir, tf, f), sep='\t', index_col=0) # .head(100)
            print(f, df2.shape)
            assert 'batch' in df2
            # print(df2.columns)
            n_rounds = len(df2.columns) - 2
            df2.columns =  list(range(n_rounds)) + ['batch', 'is_count_data']
            df2['batch'] = batch
            df2['n_rounds'] = n_rounds
            batch += 1
            df.append(df2)

# concatenation/reordering
df = pd.concat(df)
df = df[[c for c in df.columns if not c in ['batch', 'is_count_data', 'n_rounds']] + ['batch', 'is_count_data', 'n_rounds']]

ALX1-ZeroCycle_TACCAA40NTTA_0_0-TACCAA40NTTA.tsv.gz (2500, 7)
ALX1-ZeroCycle_TCCTTG40NGGT_0_0-TCCTTG40NGGT.tsv.gz (2500, 7)
ALX4-ZeroCycle_ES0_TGTGTC20NGA_0-TGTGTC20NGA.tsv.gz (2500, 7)
ALX4-ZeroCycle_TCTATT40NCAT_0_0-TCTATT40NCAT.tsv.gz (2500, 7)
ALX4-ZeroCycle_TGCGTA40NTCA_0_0-TGCGTA40NTCA.tsv.gz (2500, 7)
ALX4-ZeroCycle_TGTGTC20NGA_0_0-TGTGTC20NGA.tsv.gz (2500, 7)
CTCF-ZeroCycle_ES0_TAGCGA20NGCT_0-TAGCGA20NGCT.tsv.gz (2500, 7)
CTCF-ZeroCycle_TAGCGA20NGCT_0_0-TAGCGA20NGCT.tsv.gz (2500, 7)
EGR1-ZeroCycle_ES0_TACTAT20NATC_0-TACTAT20NATC.tsv.gz (2500, 6)
EGR1-ZeroCycle_ES0_TCTCTT20NGA_0-TCTCTT20NGA.tsv.gz (2500, 7)
EGR1-ZeroCycle_TACTAT20NATC_0_0-TACTAT20NATC.tsv.gz (2500, 6)
EGR1-ZeroCycle_TAGACA40NGTC_0_0-TAGACA40NGTC.tsv.gz (2500, 7)
EGR1-ZeroCycle_TCAGCC40NTTC_0_0-TCAGCC40NTTC.tsv.gz (2500, 7)
EGR1-ZeroCycle_TCCGCT40NGCG_0_0-TCCGCT40NGCG.tsv.gz (2500, 7)
EGR1-ZeroCycle_TCTCTT20NGA_0_0-TCTCTT20NGA.tsv.gz (2500, 7)
EGR1-ZeroCycle_TTGTAC40NGTA_0_0-TTGTAC40NGTA.tsv.gz (2500, 7)
ELF2-Zer

In [240]:
dataset = mb.datasets.SelexDataset(df, n_rounds=df['n_rounds'], labels=list(df.columns[:-3]))
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=512,
                         shuffle=False)

In [241]:
import warnings
warnings.filterwarnings('ignore')

In [242]:
import numpy as np

In [243]:
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [244]:
import gc
gc.collect()

444

In [245]:
print('total entries', dataset.rounds.shape)

total entries (182500, 7)


In [ ]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 50
log_each = 2
n_kernels = 3
lr = [0.01] * n_kernels
wd = [0.01,] + [0.001] * (n_kernels - 1)
early_stopping = [5,] + [25] * (n_kernels - 1)
w = 15 # min(len(motif), 6)
opt_kernel_shift = 1
opt_kernel_length = 1

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()

for ignore_kernel in [1]: # [0, 1]:
    print('\n\n##NEW JOB')
    print('ignore kernel', ignore_kernel)
    model_by_k, res_next = mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion,
                                                 opt_kernel_shift=opt_kernel_shift, opt_kernel_length=opt_kernel_length,
                                                 dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                 lr=wd, weight_decay=wd, ignore_kernel=ignore_kernel, use_mono=True,
                                                 num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=True, # optimiser=torch.optim.LBFGS,
                                                 n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')


print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

model = model_by_k





##NEW JOB
ignore kernel 1
next w 15 <class 'int'>
# rounds {4, 5, 7}
# batches 73
# kernels 3
# initial w 15
# enr_series True

Kernel to optimize 0

FREEZING KERNELS
optimizing feature type mono
setting grad status of kernel mono at 0 to 1
setting grad status of kernel dinuc at 0 to 0
setting grad status of kernel mono at 1 to 0
setting grad status of kernel dinuc at 1 to 0
setting grad status of kernel mono at 2 to 0
setting grad status of kernel dinuc at 2 to 0


kernels mask [0 1 1]
optimizing using <class 'torch.optim.adam.Adam'> and <class 'mubind.tl.loss.PoissonLoss'> n_epochs 50 early_stopping 5
lr= 0.01, weight_decay= 0.01, dir weight= 0
Epoch:  3, Loss: 4.609054 , best epoch: 1 secs per epoch: 7.073 s
Epoch:  5, Loss: 4.603986 , best epoch: 3 secs per epoch: 5.705 s
Epoch:  7, Loss: 4.602979 , best epoch: 5 secs per epoch: 5.258 s
Epoch:  9, Loss: 4.602657 , best epoch: 7 secs per epoch: 5.107 s
Epoch: 11, Loss: 4.602526 , best epoch: 9 secs per epoch: 5.007 s
Epoch: 13, Lo

### We can visualize the overall results obtained by the network, once the training is finished

In [ ]:
# %load_ext line_profiler
# %lprun -f mb.tl.optimize_simple mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel, num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #


In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
rcParams['figure.figsize'] = 5, 5
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1', k=5)


In [ ]:
mb.pl.conv(model)
